In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy import optimize
import shutil
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim

In [ ]:
# 0) Prepare data
#bc = datasets.load_breast_cancer()
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/car_data.csv")
print(data.head())

In [ ]:
data.info()

In [ ]:
#Convertir variables categoricas a variables numericas
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [ ]:
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [ ]:
data.info()

In [ ]:
X = data.iloc[:, 0:4]
y = data.iloc[:, 4]
m = y.size
data = data.to_numpy()

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.values.astype(np.float32))
y_test = torch.from_numpy(y_test.values.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [ ]:
# Definir el modelo del perceptrón para regresión logística
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [ ]:
# Instanciar el modelo y definir la función de pérdida y el optimizador
model = Perceptron(input_size=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Entrenamiento del modelo
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [ ]:
# Evaluación del modelo
model.eval()
with torch.no_grad():
    predicted = model(X_test)
    predicted = (predicted > 0.5).float()
    accuracy = (predicted == y_test.view(-1, 1)).sum().item() / len(y_test)
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')